In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from io import StringIO
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [22]:
def get_box_scores(link):
    link_indices = link.split('/')
    csv_name = f"{link_indices[4]}_{link_indices[5]}_{link_indices[7]}_{link_indices[8]}_{link_indices[6]}.csv"
    print(csv_name)
    driver = webdriver.Chrome()
    http_body : requests.Response = requests.get(link)
    soup_page_html : BeautifulSoup = BeautifulSoup(http_body.text, 'html.parser')
    box_scores = soup_page_html.find_all(class_="gamePod-link")
    box_score_links = ["http://ncaa.com" + box_score['href'] for box_score in box_scores]
    
    for box_score_link in box_score_links:
        print(box_score_link)
        driver.implicitly_wait(5)
        driver.get(box_score_link)
        time.sleep(5)
        driver.execute_script("window.stop();")


        game_id = box_score_link.split('/')[4]

        try:
            child_divs = driver.find_element(By.CLASS_NAME, "boxscore-team-selector").find_elements(By.TAG_NAME, "div")
        except Exception:
            continue
        child_divs_text = [div.text for div in child_divs]

        table = driver.find_element(By.CLASS_NAME, 'gamecenter-tab-boxscore').find_element(By.TAG_NAME, 'table')

        df_team_one = pd.read_html(StringIO(table.get_attribute('outerHTML')))[0]

        df_team_one['TEAM'] = child_divs_text[0]
        df_team_one['OPP'] = child_divs_text[1]
        df_team_one['GAMEID'] = game_id
        df_team_one['GAMELINK'] = box_score_link

        df_team_one = df_team_one.iloc[:-2] # Remove rows with indices 1 and 3
        
        child_divs[1].click() # Go to next tab

        table = driver.find_element(By.CLASS_NAME, 'gamecenter-tab-boxscore').find_element(By.TAG_NAME, 'table')

        df_team_two = pd.read_html(StringIO(table.get_attribute('outerHTML')))[0]

        df_team_two['TEAM'] = child_divs_text[1]
        df_team_two['OPP'] = child_divs_text[0]
        df_team_two['GAMEID'] = game_id
        df_team_two['GAMELINK'] = link

        df_team_two = df_team_two.iloc[:-2]  # Remove rows with indices 1 and 3

        # Write the first DataFrame with headers
        df_team_one.to_csv(csv_name, index=False, header=True, mode='a')

        df_team_two.to_csv(csv_name, index=False, mode='a')
        
    driver.close()


In [24]:
import datetime
from datetime import timedelta

In [26]:
yesterday = datetime.date.today() - timedelta(days = 1)

In [28]:
date = yesterday.strftime('%Y/%m/%d')

In [30]:
backfills = [
         f"https://www.ncaa.com/scoreboard/basketball-women/d1/2025/01/12/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-women/d2/2025/01/12/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-women/d3/2025/01/12/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-women/d1/2025/01/12/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-women/d2/2025/01/12/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-women/d3/2025/01/12/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-men/d1/2025/01/12/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-men/d2/2025/01/12/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-men/d3/2025/01/12/all-conf",
        ]

In [32]:
links = [
         f"https://www.ncaa.com/scoreboard/basketball-men/d1/{date}/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-men/d2/{date}/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-men/d3/{date}/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-women/d1/{date}/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-women/d2/{date}/all-conf",
         f"https://www.ncaa.com/scoreboard/basketball-women/d3/{date}/all-conf"]

In [17]:
for link in backfills:
    get_box_scores(link)

basketball-women_d1_01_12_2025.csv
http://ncaa.com/game/6356699


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x000000010533a138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x0000000105332988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x0000000104d98968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000104d9d84c cxxbridge1$string$len + 109424
4   chromedriver                        0x0000000104d9fb3c cxxbridge1$string$len + 118368
5   chromedriver                        0x0000000104e16d44 cxxbridge1$string$len + 606312
6   chromedriver                        0x0000000104e164f0 cxxbridge1$string$len + 604180
7   chromedriver                        0x0000000104dd1564

In [34]:
for link in links:
    get_box_scores(link)

basketball-men_d1_01_13_2025.csv
http://ncaa.com/game/6354751
http://ncaa.com/game/6352148
http://ncaa.com/game/6351654
http://ncaa.com/game/6349434


TimeoutException: Message: timeout: Timed out receiving message from renderer: 298.464
  (Session info: chrome=131.0.6778.266)
Stacktrace:
0   chromedriver                        0x00000001033d2138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x00000001033ca988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x0000000102e30968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000102e1bc90 cxxbridge1$string$len + 4020
4   chromedriver                        0x0000000102e1ba00 cxxbridge1$string$len + 3364
5   chromedriver                        0x0000000102e1978c core::str::slice_error_fail::ha0e52dbcb60e6bae + 59404
6   chromedriver                        0x0000000102e1a3f8 core::str::slice_error_fail::ha0e52dbcb60e6bae + 62584
7   chromedriver                        0x0000000102e28000 cxxbridge1$string$len + 54052
8   chromedriver                        0x0000000102e3bb28 cxxbridge1$string$len + 134732
9   chromedri